In [1]:
# Section 1 - Imports
# -------------------

# imports
import datetime
import attotime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import math
from os.path import join
import copy

# DSO modules
from dsoclasses.doris.algorithms import beacon_nominal_frequency
from dsoclasses.rinex.doris.rinex import DorisRinex
from dsoclasses.time.pyattotime import at2pt, fsec2asec, to_attoseconds
from dsoclasses.orbits import sp3c, interpolator
from dsoclasses.geodesy import transformations
from dsoclasses.gnss import systems as gs
from dsoclasses.gnss import algorithms as alg
from dsoclasses.troposphere.vmf3 import SiteVmf3
from dsoclasses.sinex import sinex

In [2]:
# Section 2.1 – Define Files needed for analysis
# -----------------------------------------------
data_path = "/home/xanthos/Software/AcademicSoftware/data"
drinex = join(data_path, "s6arx24001.001")
dpod = join(data_path, "dpod2020_041.snx")
dpod_freq_corr = join(data_path, "dpod2020_041_freq_corr.txt")
dsp3 = join(data_path, "ssas6a20.b23357.e24001.DG_.sp3.001")
vmf3_data = join(data_path, "y2024.vmf3_d")

In [3]:
# Section 2.2 - Load Orbit Data + Initialize Interpolator
# -------------------------------------------------------
intrp = interpolator.Sp3Interpolator.from_sp3(dsp3, ['L'], interval_in_sec=310, min_data_pts=10, itype='Barycentric')

In [4]:
# Section 2.3 - Intitialize DorisRinex and select a beacon (Dionysos/DIOB)
# ------------------------------------------------------------------------

rnx = DorisRinex(drinex)

site_name = 'DIOB'
result = sinex.extract_sinex_coordinates(dpod, [site_name], rnx.time_first_obs, True, dpod_freq_corr)
rsta = np.array([result[site_name]['X'], result[site_name]['Y'], result[site_name]['Z']])

# we are also going to need the geodetic coordinates of the site
lat, lon, hgt = transformations.car2ell(*rsta)

In [5]:
# Section 2.4 - Initialize VMF3 from (DORIS) site data
# ----------------------------------------------------
vmf = SiteVmf3(vmf3_data, [site_name])

In [10]:
# Section 4 – Preprocess Doppler observations Iono-Free LC
# --------------------------------------------------------

C = 299792458e0 # speed of light in vacum, [m/sec]

# nominal transmitting frequencies for DIOB
s1, u2 = beacon_nominal_frequency(rnx.kfactor(site_name))
feN = frN = (gamma * s1 - sqrt_gamma * u2) / (gamma-1.)
def ifcorr(L2ghz, L400mhz):
    return (s1*s1*L2ghz - u2*u2*L400mhz) / (s1*s1 - u2*u2)
print(f'Nominal frequencies S1={s1*1e-6:.1f}GHz (or {C/s1:.3f} m.), U2={u2*1e-3:.1f}MHz (or {C/u2:.3f} m.)')
print(f'Iono-Free LC frequency L3={feN*1e-6:.1f}GHz or {C/feN:.3f} m.')
assert feN == frN + 1

# Apply PCO to 2GHz phase center at beacon
rsta = rsta + transformations.geodetic2lvlh(lat, lon) @ np.array([0e0, 0e0, 487e-3])

# store results here
diono = []; epochs = []; elevations = [];

# for every block in the RINEX file
for block in rnx:
    # for every beacon in the block
    for beacon, data in block:
        # match DIOB
        if beacon == rnx.name2id(site_name):
            
            # use the block-provided clock correction to get to (approximate) TAI
            # date(TAI) = epoch + receiver clock offset
            # this is the actual epoch of observation
            tai = block.t() + attotime.attotimedelta(nanoseconds=block.clock_offset() * 1e9)
            
            # Satellite position at signal emission time; store into an array
            satx, saty, satz, _, _ = alg.sat_at_emission_time(rsta, tai, intrp, 'L40')
            rsat = np.array([satx, saty, satz])

            # compute elevation
            _, _, el = transformations.azele(rsat, rsta)
    
            # Phase measurements at 2GHz carrier
            L2ghz = data['L1']['value']
            L400mhz = data['L2']['value']
            
            diono.append(ifcorr(L2ghz, L400mhz))
            epochs.append(tai)
            elevations.append(np.degrees(el))

plt.scatter([at2pt(ti) for ti in epochs], diono, alpha=0.5, s=12, c='red')
plt.grid(color='0.95')
plt.ylabel(f"Residuals from L1 Range-Rate at {site_name} in [m/sec]")
plt.title(f"Satellite {rnx.sat_name}")
# Format the datetime x-axis
ax = plt.gca()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%m-%y %H:%M:%S"))
# auto-rotate date labels for readability
plt.gcf().autofmt_xdate()
plt.show()

Nominal frequencies S1=2036.2GHz (or 0.147 m.), U2=401250.0MHz (or 0.747 m.)
Iono-Free LC frequency L3=2036.2GHz or 0.147 m.


AssertionError: 